## Compile the Projects
To-do
* Figure out how to version things b/c projects will get updated and we want to track any changes.
* This only needs to be done with data from lp2000 and ctips.
* Need to track changes across all the different dataframes
* Use merges to figure it out?

In [1]:
import hashlib
from datetime import datetime

import _harmonization_utils as har_utils
import _lrtp_utils as lrtp_utils
import _sb1_utils as sb1_utils
import _specific_list_utils
import _state_rail_plan_utils as srp_utils
import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_list/"

## Split the dataframes/Create an ID

### LRTP/LOST

In [4]:
lrtp_lost_df, lrtp_lost_gdf = lrtp_utils.all_mpo(True)

/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

96 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:720: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sandag.cost2020m.str.replace("$", "")
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


65 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


360 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()


68 rows are headers


/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_lrtp_utils.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract

In [5]:
def database_unique_project_number(
    df: pd.DataFrame, project_id_col: str, database: str
) -> pd.DataFrame:

    # df['combo'] = df.apply(lambda row: f"{row.name}{row['timestamp']}{''.join(map(str, row))}", axis=1)
    # df['project_number'] = df['combo'].apply(lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:12])
    # df = df.drop(columns = ['combo', 'timestamp'])

    df["project_number"] = df[project_id_col].astype(str).replace("(", "_").replace(
        ")", "_"
    ) + df[database].str.replace(" ", "_")
    # Pad the number
    df["project_number"] = df["project_number"].str.pad(width=14, fillchar="0")

    # df = df.drop(columns = [project_id_col])
    return df

In [6]:
def non_database_unique_project_number(
    df: pd.DataFrame, data_source_col: str
) -> pd.DataFrame:

    df["project_count"] = df.groupby([data_source_col]).cumcount().astype(int) + 1
    # Pad the number
    df["project_number"] = df[data_source_col].str.replace(
        " ", "_"
    ) + df.project_count.astype(str)

    df.project_number = df.project_number.str.pad(width=14, fillchar="0")

    df = df.drop(columns=["project_count"])
    return df

In [7]:
def separate_out_df(df: pd.DataFrame, columns_to_keep: list) -> pd.DataFrame:
    """
    Subset the column, drop any rows
    in which the values are Nan or "None."
    """
    # Subset
    df2 = df[columns_to_keep]

    # Fill in missing values
    try:
        df2 = df2.fillna("none")
    except:
        df2

    # Remove project_number and keep only cols of interest
    columns_to_keep.remove("project_number")

    # Drop rows that are nan or "None" based on how many columns are listed
    if len(columns_to_keep) == 1:
        df2 = df2.dropna(how="any")
        df2 = df2[
            df2.applymap(lambda x: x.lower() if isinstance(x, str) else x) != "none"
        ].dropna()

    # If there are more than one column to separate out,
    # keep any row that has a non-null value
    else:
        df2 = df2.dropna(how="all", subset=columns_to_keep)
    return df2

In [8]:
lrtp_lost_df = non_database_unique_project_number(lrtp_lost_df, "data_source")

In [9]:
lrtp_lost_df.sample()

,project_title,lead_agency,project_year,project_description,total_project_cost,geometry,city,county,data_source,notes,project_number
516,Marks: Malaga To American,Fresno Rtp/Scs Published In 2022,2042-2046,Ac Overlay,317000.00,None,None,None,Fresno Rtp/Scs Published In 2022 Lrtp,"Project Type: Streets & Roads-Maintenance, Financial Constraint: Unconstrained",Fresno_Rtp/Scs_Published_In_2022_Lrtp517


In [10]:
lrtp_lost_county = separate_out_df(lrtp_lost_df, ["project_number", "county"])

In [11]:
lrtp_lost_county.sample()

,project_number,county
1317,000000Lost1318,San Diego


In [12]:
lrtp_lost_city = separate_out_df(lrtp_lost_df, ["project_number", "city"])

In [13]:
lrtp_lost_city.shape

(745, 2)

In [14]:
lrtp_lost_agency = separate_out_df(lrtp_lost_df, ["project_number", "lead_agency"])

In [15]:
lrtp_lost_agency.sample()

,project_number,lead_agency
959,Scrtpa_Rtp/Scs_Published_In_2018_Lrtp925,Scrtpa Rtp/Scs Published In 2018


In [16]:
lrtp_lost_geo = separate_out_df(lrtp_lost_df, ["project_number", "geometry"])

In [17]:
lrtp_lost_geo.shape, lrtp_lost_gdf.shape

((1357, 2), (1355, 10))

In [18]:
lrtp_to_drop = ["county", "city", "lead_agency", "geometry"]

In [19]:
lrtp_lost_df = lrtp_lost_df.drop(columns=lrtp_to_drop)

In [20]:
lrtp_lost_df.sample()

,project_title,project_year,project_description,total_project_cost,data_source,notes,project_number
1876,No Title,2021,"In Western Riverside County For Rta: Associated Transit Improvements, Including But Not Limited To Bus Shelters And Benches, Kiosks, Signage, And Lighting To Enhance Security And Safety And Ada Access (Fy 18 5307 Uza: Riv-Sb $10 And Mt $30)",50000.00,Scag Rtp/Scs Published In 2020 Lrtp,"System: Transit, Route #: 0, Route Name: Nan, From: Nan, To: Nan",Scag_Rtp/Scs_Published_In_2020_Lrtp1877


### SB1
* Not all projects have a project ID...have to come up with our own?

In [21]:
sb1_df = sb1_utils.load_sb1()

/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gdf[i]
/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  gdf[i]


0 rows are mising geometry
7917 rows contain invalid geography


In [22]:
#  sb1_df['data_source'] = "SB1 Server"

In [23]:
sb1_df = non_database_unique_project_number(sb1_df, "data_source")

In [24]:
len(sb1_df)

9186

In [25]:
sb1_df.project_number.nunique()

9186

#### Check that assemblydistrict and assemblycode are the same values

In [26]:
sb1_df["assembly_same"] = sb1_df.assemblycode == sb1_df.assemblydistrict

In [27]:
sb1_df.assembly_same.value_counts()

True     7217
False    1969
Name: assembly_same, dtype: int64

In [28]:
sb1_df["senate_same"] = sb1_df.senatedistrict == sb1_df.senatecode

In [29]:
sb1_df.senate_same.value_counts()

True     6950
False    2236
Name: senate_same, dtype: int64

In [30]:
sb1_county = separate_out_df(sb1_df, ["project_number", "countyname"])

In [31]:
def explode_dataframe(df: pd.DataFrame, column_to_explode: str) -> pd.DataFrame:
    df["Column2"] = df[column_to_explode].apply(
        lambda x: [int(i) if i.isdigit() else i for i in x.replace(",", "").split()]
    )
    df = df.drop(columns=[column_to_explode])
    df2 = df.explode("Column2")
    df2 = df2.rename(columns={"Column2": column_to_explode})
    return df2

In [32]:
sb1_assembly = separate_out_df(sb1_df, ["project_number", "assemblydistrict"])

In [33]:
sb1_assembly = explode_dataframe(sb1_assembly, "assemblydistrict")

In [34]:
sb1_senate = separate_out_df(sb1_df, ["project_number", "senatedistrict"])

In [35]:
sb1_senate = explode_dataframe(sb1_senate, "senatedistrict")

In [36]:
sb1_city = separate_out_df(sb1_df, ["project_number", "cityname"])

In [37]:
sb1_geo = separate_out_df(sb1_df, ["project_number", "geometry"])

In [38]:
sb1_awards = separate_out_df(sb1_df, ["project_number", "projprogram"])

In [39]:
sb1_awards.shape, sb1_geo.shape, sb1_city.shape

((9186, 2), (1585, 2), (6696, 2))

In [40]:
sb1_agencies = separate_out_df(
    sb1_df, ["project_number", "projagency", "appagencyname", "impagencyname"]
)

In [41]:
sb1_agencies.shape

(9186, 4)

In [42]:
sb1_agencies.project_number.nunique()

9186

In [43]:
sb1_df.project_number.value_counts().head()

000SB1_Server1    1
SB1_Server6127    1
SB1_Server6121    1
SB1_Server6122    1
SB1_Server6123    1
Name: project_number, dtype: int64

In [44]:
sb1_counties = separate_out_df(sb1_df, ["project_number", "countyname"])

In [45]:
sb1_counties.sample(3)

,project_number,countyname
2755,SB1_Server6082,Merced
3201,SB1_Server8077,Imperial
8144,SB1_Server3232,Santa Barbara


In [139]:
sb1_df.columns

Index(['projectid', 'projname', 'projcatcode', 'projcategory', 'projprogcode',
       'projprogram', 'multiprogfunded', 'projstatus', 'description', 'cost',
       'assemblydistrict', 'senatedistrict', 'assemblycode', 'senatecode',
       'countyname', 'cityname', 'countycode', 'citycode', 'appagencyname',
       'impagencyname', 'geometry', 'totalcosts', 'routes', 'constyear',
       'costfull', 'projagency', 'data_source', 'project_number',
       'assembly_same', 'senate_same'],
      dtype='object')

### LP2000

In [46]:
def load_lp2000(file: str):

    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    ).drop(columns=["project_label_name"])

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    ).drop(columns=["project_label_name"])

    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )

    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )

    df_project["data_source"] = "lp2000"
    # Add timestamp column
    return df_project, df_county, df_district, df_award, df_phase

In [47]:
(
    lp2000_project,
    lp2000_county,
    lp2000_district,
    lp2000_award,
    lp2000_phase,
) = load_lp2000("LP2000_projects.xlsx")

In [48]:
lp2000_project = database_unique_project_number(
    lp2000_project, "project_id", "data_source"
)

In [49]:
len(lp2000_project), lp2000_project.project_number.nunique()

(11272, 11272)

In [50]:
lp2000_agencies = lp2000_project[["project_number", "agency_name"]]

In [51]:
lp2000_project = lp2000_project.drop(columns=["agency_name"])

In [52]:
lp2000_project.sample()

,project_id,comment_desc,est_total_prj_costs,location_name,project_label_name,original_post_mile_begin_id,original_post_mile_end_id,revised_post_mile_begin_ind,revised_post_mile_end_ind,route_name,state_hwy_ind,senate_district_code,update_date_time,urban_area_code,work_type_desc,category_desc,current_phase,active_transportation_program__atp_,bridge_inspection___scour_evaluation,covid_relief_funds_for_highway_infrastructure_programs_for_stip_covid_augmentation,carbon_reduction_program__crp_,congestion_mitigation___air_quality_improvement_program__cmaq_,coronavirus_response_and_relief_supplemental_appropriations_act__crrsaa__funds,corridor_mobility_improvement_account__cmia__program,county_exchange_funds,county_state_match_program,"earmarks_projects__hpp,_demo_cpfcds,_etc__",emergency_relief__er_,ferry_boat_program__fbp__and_ferry_boat_discretionary__fbd__program,"funds_for_planning,_programming_and_monitoring___rip",general_funded_designated_programs,hazard_elimination_safety__hes_,high_risk_rural_roads_program__hr3_,highway_bridge_,highway_safety_improvement_program__hsip___infrastructure__state_fund,highway_safety_improvement_program__hsip___non_infrastructure_,highway_safety_improvement_program__hsip__infrastructure__federal_fund,local_partnership_program__lpp_–_competitive__,local_roads,local_roads_rehabilitation,railroad_grade_crossing_protection,railroad_grade_separations,"rebuilding_american_infrastructure_with_sustainability_and_equity__raise__and_multimodal_project_discretionary_grant_programs__e_g_,_infra,_mega,_rstg_or_rural__",regional_improvement_program_–_regional_share_of_stip_transportation_enhancement__off_system_,regional_surface_transportation_block_grant_program__rstbgp__and_highway_infrastructure_program__hip_,regional_transportation_planning_agency__rtpa__stp_match_exchange,sb1_funded_freeway_service_patrol,shopp__traffic_light_synchronization_program__tlsp___proposition_1b_bond_funds,safe_routes_to_school__sr2s_and_srts_,set_aside_coordinated_border_infrastructure__cbi__program_under_fast_act,solutions_for_congested_corridors_program__sccp_,special_programs,state_local_partnership_program__slpp__and_local_partnership_program__lpp_formulaic_,structures_seismic_retrofit_,trade_corridor_enhancement_account__tcea__programs_–_local_share,trade_corridor_enhancement_account__tcea__programs_–_state_share,trade_corridors_improvement_fund__tcif__program_local_streets___roads,traffic_congestion_relief_program___tcrp__,data_source,project_number
8725,5160(028),CTIPS: 205-0000-0303\nDBE = 15% due to specialized work,NaN,Los Banos SR-152 Signals Synchronization,Signal Synchronization,NaN,NaN,NaN,NaN,0-LBNS,N,NaN,2023-12-22 09:55:35,3042.00,NaN,Signal Synchronization,single phase,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,lp2000,5160(028)lp2000


In [53]:
def add_project_number(
    df_with_project_num: pd.DataFrame, target_df: pd.DataFrame, merge_col: str
) -> pd.DataFrame:
    df_with_project_num_subset = df_with_project_num[["project_number"] + [merge_col]]
    m1 = pd.merge(df_with_project_num_subset, target_df, on=merge_col, how="inner")
    m1 = m1.drop(columns=merge_col)
    return m1

In [54]:
lp2000_county = add_project_number(lp2000_project, lp2000_county, "project_id")

In [55]:
lp2000_county.sample()

,project_number,county_name
8303,15A5(040)lp2000,Butte County


In [56]:
lp2000_district = add_project_number(lp2000_project, lp2000_district, "project_id")

In [57]:
lp2000_district.sample()

,project_number,district_code
9858,6498(005)lp2000,3.00


In [58]:
lp2000_award = add_project_number(lp2000_project, lp2000_award, "project_id")

In [59]:
lp2000_award.sample()

,project_number,grant_program,state_fiscal_awarded_year
3310,5927(129)lp2000,Local Assistance,2021


In [60]:
lp2000_phase = add_project_number(lp2000_project, lp2000_phase, "project_id")

In [61]:
lp2000_phase.sample()

,project_number,single_phase_cost,single_phase_expenditure_amt,total_state_funds,total_federal_funds,is_state,is_federal
2104,5933(159)lp2000,504621.00,46330.61,0.00,504621.00,No,Yes


### CTIPS

In [62]:
def load_ctips(file: str):

    df_project = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="project")
    )

    df_agency = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="agencies")
    )

    df_county = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="county")
    )

    df_district = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="district")
    )

    df_phase = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="phase_funding")
    )

    df_award = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="awards")
    )

    df_house = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="us_house")
    )

    df_senate = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="senate")
    )

    df_assembly = to_snakecase(
        pd.read_excel(f"{GCS_FILE_PATH}LP2000_CTIPS/{file}", sheet_name="assembly")
    )

    df_project["data_source"] = "CTIPS"
    return (
        df_project,
        df_agency,
        df_county,
        df_district,
        df_phase,
        df_award,
        df_house,
        df_senate,
        df_assembly,
    )

In [63]:
(
    ctips_project,
    ctips_agencies,
    ctips_county,
    ctips_district,
    ctips_phase,
    ctips_award,
    ctips_house,
    ctips_senate,
    ctips_assembly,
) = load_ctips("CTIPS_data.xlsx")

In [64]:
ctips_project = database_unique_project_number(ctips_project, "ctips_id", "data_source")

In [65]:
ctips_agencies = add_project_number(ctips_project, ctips_agencies, "ctips_id")

In [66]:
ctips_county = add_project_number(ctips_project, ctips_county, "ctips_id")

In [67]:
ctips_district = add_project_number(ctips_project, ctips_district, "ctips_id")

In [68]:
ctips_phase = add_project_number(ctips_project, ctips_phase, "ctips_id")

In [69]:
ctips_award = add_project_number(ctips_project, ctips_award, "ctips_id")

In [70]:
ctips_house = add_project_number(ctips_project, ctips_house, "ctips_id")

In [71]:
ctips_senate = add_project_number(ctips_project, ctips_senate, "ctips_id")

In [72]:
ctips_assembly = add_project_number(ctips_project, ctips_assembly, "ctips_id")

In [73]:
ctips_award.sample()

,project_number,line_year,program,progdesc
205,20950001262CTIPS,2024,Local Funds,None


In [74]:
ctips_assembly.sample()

,project_number,assembly
2216,10800000055CTIPS,35


#### Have to 

In [75]:
ctips_agencies.sample()

,project_number,agency_name,implpaed_agency,implpse_agency,implrw_agency,implcon_agency
8606,11300000393CTIPS,Caltrans,NaN,NaN,NaN,NaN


### State Rail Plan

In [76]:
srp_df = har_utils.load_state_rail_plan()

In [77]:
srp_df = non_database_unique_project_number(srp_df, "data_source")

In [78]:
srp_df.sample()

,project_time_horizon,project_name,project_description,lead_agency,total_project_cost,srp_region,corridor,sub_corridor_node_1,sub_corridor_node_2,itsp_corridor,project_category,data_source,project_number
107,Near Term,Riverside Line Service Improvement & Capacity Study,Initial project work to develop plan for operating additional service on host railroad property.,Metrolink,541000,NaN,NaN,NaN,NaN,NaN,Capital,State Rail Plan,State_Rail_Plan108


In [79]:
srp_df_agency = separate_out_df(srp_df, ["project_number", "lead_agency"])

In [80]:
srp_df = srp_df.drop(columns=["lead_agency"])

## Concat/Harmonize

### Agencies
* lrtp_lost_agency
* sb1_agencies
* lp2000_agencies
* ctips_agencies
* srp_df_agency

* Agency should be attached back to projects...

In [81]:
lrtp_lost_agency.sample()

,project_number,lead_agency
46,Sandag_Rtp/Scs_Published_2021_Lrtp47,Sandag Rtp/Scs Published 2021


In [82]:
sb1_agencies.sample()

,project_number,projagency,appagencyname,impagencyname
2211,0SB1_Server289,Colton City,none,none


In [83]:
lp2000_agencies.sample()

,project_number,agency_name
8752,5157(125)lp2000,Madera


In [84]:
ctips_agencies.sample()

,project_number,agency_name,implpaed_agency,implpse_agency,implrw_agency,implcon_agency
19717,20920012146CTIPS,"Los Angeles, City of",NaN,NaN,NaN,NaN


In [85]:
srp_df_agency.sample()

,project_number,lead_agency
47,State_Rail_Plan48,Caltrans


In [86]:
def harmonize_agencies(df: pd.DataFrame, agency_column: str) -> pd.DataFrame:
    df = df.rename(columns={agency_column: "lead_agency"})

    return df

In [87]:
srp_df_agency = harmonize_agencies(srp_df_agency, "lead_agency")

In [88]:
ctips_agencies = harmonize_agencies(ctips_agencies, "lead_agency")

### Assembly_districts
* ctips_assembly
* sb1_assembly
* Find the assembly district if we have the coordinates?

In [89]:
ctips_assembly.sample()

,project_number,assembly
97,11300000025CTIPS,33


In [90]:
sb1_assembly.sample()

,project_number,assemblydistrict
1257,SB1_Server9089,65


In [91]:
def harmonize_assembly(df: pd.DataFrame, assembly_column: str) -> pd.DataFrame:
    df = df.rename(columns={assembly_column: "assembly_districts"})

    return df

In [92]:
ctips_assembly = harmonize_assembly(ctips_assembly, "assembly")

In [93]:
sb1_assembly = harmonize_assembly(sb1_assembly, "assemblydistrict")

In [94]:
all_assembly_df = pd.concat([ctips_assembly, sb1_assembly])

In [95]:
all_assembly_df.head()

,project_number,assembly_districts
0,10700000048CTIPS,5
1,10700000048CTIPS,6
2,10900000289CTIPS,36
3,10900000289CTIPS,37
4,10900000289CTIPS,38


In [96]:
all_assembly_df.project_number.nunique(), len(all_assembly_df)

(12409, 17423)

### Awards
* sb1_awards
* lp2000_award
* ctips_award

In [97]:
sb1_awards.sample()

,project_number,projprogram
8800,SB1_Server7065,State Highway Projects


In [98]:
lp2000_award.sample()

,project_number,grant_program,state_fiscal_awarded_year
3395,5940(152)lp2000,Local Assistance,2122


In [99]:
ctips_award.sample()

,project_number,line_year,program,progdesc
23715,20100000357CTIPS,2004,Local Funds,None


In [100]:
def harmonize_awards(
    df: pd.DataFrame, year: str, grant_program_col: str
) -> pd.DataFrame:
    rename_dict = {year: "award_year", grant_program_col: "grant_program"}
    df = df.rename(columns=rename_dict)

    create_cols = ["award_year", "grant_program"]
    for column in create_cols:
        if column not in df:
            df[column] = "Not Available"
    return df

In [101]:
sb1_awards = harmonize_awards(sb1_awards, "", "projprogram")

In [102]:
lp2000_award = harmonize_awards(
    lp2000_award, "state_fiscal_awarded_year", "grant_program"
)

In [103]:
ctips_award = harmonize_awards(ctips_award, "line_year", "program")

In [104]:
all_awards_df = pd.concat([sb1_awards, lp2000_award, ctips_award])

In [105]:
all_awards_df.shape

(58937, 4)

In [106]:
all_awards_df.project_number.nunique()

40434

In [107]:
all_awards_df.grant_program.unique()

array(['Trade Corridor Enhancement Program',
       'Solutions For Congested Corridors Program',
       'Local Partnership Program',
       'Solutions To Congested Corridors Program',
       '201718 Local Streets And Roads', 'State Highway Projects',
       '201819 Local Streets And Roads', 'Active Transportation Program',
       'State Transit Assistance', 'State Of Good Repair',
       'Transit And Intercity Rail Capital Program',
       'State Rail Assistance', 'Local Streets And Roads',
       'Local Assistance', 'Active Transportation Program (ATP)',
       'Proposition 1B, Hwy Safety, Traffic Reduction, Air Quality , and Port Security Bond Act of 2006, and SB 1: The Road Repair and Accountability Act of 2017',
       'State Transportation Improvement Program (STIP)',
       'Traffic Congestion Relief Program – (TCRP)',
       'Coordinated Border Infrastructure (CBI) Program', 'State SB1',
       'RSTP', 'RIP', 'Other State', 'Other Fed', 'Local Funds',
       'Future Need', 'Fede

### Cities
* lrtp_lost_city
* sb1_city
* Need to find a way to retain Daly City

In [108]:
lrtp_lost_city.sample()

,project_number,city
601,0000000Lost602,City Of Brawley


In [109]:
sb1_city.sample()

,project_number,cityname
5861,SB1_Server1409,Anaheim Submitted By City


In [118]:
def harmonize_cities(df: pd.DataFrame, city_column: str) -> pd.DataFrame:
    df = df.rename(columns={city_column: "city"})

    df.city = (
        df.city.str.title()
        .str.replace("City Of", "")
        .str.replace("Submitted By", "")
        .str.strip()
    )

    return df

In [119]:
sb1_city = harmonize_cities(sb1_city, "cityname")

In [120]:
lrtp_lost_city = harmonize_cities(lrtp_lost_city, "city")

In [121]:
all_city_df = pd.concat([sb1_city, lrtp_lost_city])

In [123]:
all_city_df[["city"]].drop_duplicates().sort_values(by=["city"])

,city
247,
3929,Adelanto
605,Adelanto Apple Valley Barstow Hesperia Victorville
21,Adelanto Victorville
4076,Agoura Hills
552,Agoura Hills Alhambra Arcadia Artesia Azusa Baldwin Park Bell Bell Gardens Bellflower Beverly Hills Bradbury Burbank Calabasas Carson Cerritos Commerce Compton Cudahy Culver Downey Duarte El Monte El Segundo Gardena Glendale
8994,Agoura Hills Calabasas Hidden Hills Los Angeles
1900,Alameda
631,Alameda Albany Berkeley El Cerrito Emeryville Fremont Hayward Newark Oakland Piedmont Pinole Richmond San Leandro San Pablo Union
1665,Alameda Albany Berkeley Emeryville Fremont Hayward Newark Oakland Piedmont San Leandro Union El Cerrito Pinole Richmond San Pablo


### Counties
* lrtp_lost_county
* sb1_counties
* lp2000_county
* ctips_county
* Clean the counties too, find a way to separate out counties separated out by spaces

In [125]:
lrtp_lost_county.sample()

,project_number,county
1246,000000Lost1247,San Bernardino


In [126]:
sb1_counties.sample()

,project_number,countyname
2152,SB1_Server7564,Riverside


In [127]:
lp2000_county.sample()

,project_number,county_name
1350,5006(872)lp2000,Los Angeles County


In [128]:
ctips_county.sample()

,project_number,county
6024,20920002147CTIPS,Los Angeles County


In [129]:
def harmonize_counties(df: pd.DataFrame, county_column: str) -> pd.DataFrame:
    rename_columns = {
        county_column: "county",
    }

    df = df.rename(columns=rename_columns)

    df.county = (
        df.county.str.title().str.replace("County", "").str.replace("Submitted By", "")
    )

    return df

In [130]:
ctips_county = harmonize_counties(ctips_county, "county")

In [131]:
lp2000_county = harmonize_counties(lp2000_county, "county_name")

In [132]:
sb1_counties = harmonize_counties(sb1_counties, "countyname")

In [133]:
lrtp_lost_county = harmonize_counties(lrtp_lost_county, "county")

In [134]:
all_counties_df = pd.concat(
    [ctips_county, lp2000_county, sb1_counties, lrtp_lost_county]
)

In [135]:
all_counties_df.shape

(51584, 2)

In [136]:
all_counties_df.project_number.nunique()

51484

In [137]:
all_counties_df.county.unique()

array(['Various Counties', 'Orange ', 'San Bernardino ', 'San Joaquin ',
       'Riverside ', 'Los Angeles ', 'Ventura ', 'Fresno ', 'San Diego ',
       'Yolo ', 'Monterey ', 'Merced ', 'Santa Barbara ', 'Stanislaus ',
       'Madera ', 'Shasta ', 'El Dorado ', 'Kern ', 'Sacramento ',
       'Yuba ', 'Modoc ', 'Placer ', 'Humboldt ', 'Sutter ', 'Mendocino ',
       'Tuolumne ', 'San Luis Obispo ', 'Santa Clara ', 'Mariposa ',
       'Santa Cruz ', 'Alpine ', 'Plumas ', 'Mono ', 'San Francisco ',
       'Nevada ', 'Tulare ', 'Lake ', 'San Mateo ', 'Solano ', 'Alameda ',
       'Contra Costa ', 'Sonoma ', 'San Benito ', 'Tehama ', 'Napa ',
       'Calaveras ', 'Marin ', 'Kings ', 'Sierra ', 'Amador ',
       'Imperial ', 'Trinity ', 'Inyo ', 'Butte ', 'Lassen ', 'Colusa ',
       'Del Norte ', 'Douglas , Nev', 'Siskiyou ', 'Glenn ',
       'Washoe , Nev', 'Carson City , Nev', 'Multi-', nan, 'Alameda',
       'Santa Barbara', 'Los Angeles', 'Merced Stanislaus', 'Shasta',
       'Imperial

### Congressional Districts
* None of the datasets have congressional district info

### CT Districts
* Manually fill it in for LRTP/LOST? 
* Missing this column in SB1 but it shouldn't be...
* lp2000_district
* ctips_district

In [140]:
ctips_district.sample()

,project_number,districtid
13246,20930001161CTIPS,12


In [141]:
lp2000_district.sample()

,project_number,district_code
8431,5030(067)lp2000,4.00


In [143]:
def harmonize_ct_districts(df: pd.DataFrame, district_column: str) -> pd.DataFrame:
    df = df.rename(columns={district_column: "ct_districts"})

    return df

In [144]:
lp2000_district = harmonize_ct_districts(lp2000_district, "district_code")

In [145]:
ctips_district = harmonize_ct_districts(ctips_district, "districtid")

In [146]:
all_ct_dist_df = pd.concat([lp2000_district, ctips_district])

### Geometry
* lrtp_lost_gdf
* sb1_geo
* 

In [150]:
lrtp_lost_gdf.columns

Index(['project_title', 'lead_agency', 'project_year', 'project_description',
       'total_project_cost', 'geometry', 'city', 'county', 'data_source',
       'notes'],
      dtype='object')

In [156]:
sb1_geo.columns

Index(['project_number', 'geometry'], dtype='object')

In [154]:
def harmonize_geo(gdf: gpd.GeoDataFrame, geography_col: str) -> gpd.GeoDataFrame:
    gdf = gdf.rename(columns={geography_col: "geometry"})
    # Add CRS
    gdf = gdf.set_geometry('geometry')
    return gdf

In [155]:
lrtp_lost_gdf = harmonize_geo(lrtp_lost_gdf, "geometry")

In [157]:
sb1_geo = harmonize_geo(sb1_geo, "geometry")

In [158]:
all_geo_df = pd.concat([lrtp_lost_gdf, sb1_geo])

In [159]:
type(all_geo_df)

geopandas.geodataframe.GeoDataFrame

In [160]:
len(all_geo_df)

2940

### Phase Funding
* ctips_phase
* lp2000_phase

In [161]:
lp2000_phase.sample()

,project_number,single_phase_cost,single_phase_expenditure_amt,total_state_funds,total_federal_funds,is_state,is_federal
2349,5441(077)lp2000,138600.00,0.00,138600.00,0.00,Yes,No


In [162]:
ctips_phase.sample()

,project_number,con,rw,pe_env,pe_rw,pe_con,pe_total,total_federal_funds,total_state_funds,total_local_funds,is_local,is_state,is_federal,projectid,pa_ed_begin,pa_ed_end,ps_e_begin,begin_row,end_row,con_start_date,con_end_date,begin_closeout,end_closeout,construction_completion_date,ready_to_list_date,projcomp_date
11372,10900003619CTIPS,1796000.00,10000.00,274000.00,10000.00,310000.00,0.00,2460000.00,0.00,0.00,No,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def harmonize_ct_districts(df: pd.DataFrame, district_column: str) -> pd.DataFrame:
    df = df.rename(columns={district_column: "ct_districts"})

    return df

### Project

### Senate District